In [ ]:
from arcgis.gis import GIS
from arcgis.raster.functions import apply
from IPython.display import Image

In [ ]:
gis = GIS('https://ionutsandric.maps.arcgis.com', 'ionutsandric', 'icub2018')

In [ ]:
landsat_item = gis.content.search('Multispectral Landsat', 'Imagery Layer', outside_org=True)[0]

In [ ]:
sentinel_item = gis.content.search('Multispectral Sentinel-2', 'Imagery Layer', outside_org=True)[0]

In [ ]:
# View imagery properties
from IPython.display import HTML

In [ ]:
# Access and use layers from Image Services
landsat = landsat_item.layers[0]

In [ ]:
sentinel = sentinel_item.layers[0]

In [ ]:
# Explore wavelengths
import pandas as pd

In [ ]:
pd.DataFrame(landsat.key_properties()['BandProperties'])

In [ ]:
pd.DataFrame(sentinel.key_properties()['BandProperties'])

In [ ]:
# Vizualize layers in map

In [ ]:
m = gis.map('Bucharest, Romania')
m

In [ ]:
for rasterfunc in landsat.properties.rasterFunctionInfos:
    print(rasterfunc.name)

In [ ]:
for rasterfunc in sentinel.properties.rasterFunctionInfos:
    print(rasterfunc.name)

In [ ]:
# Raster function for NDVI
from arcgis.geocoding import geocode
area = geocode('Bucharest, Romania', out_sr=landsat.properties.spatialReference)[0]

In [ ]:
landsat.extent = area['extent']

In [ ]:
sentinel.extent = area['extent']

In [ ]:
# Custom bands

In [ ]:
from arcgis.raster.functions import stretch, extract_band

In [ ]:
naturalcolor = stretch(extract_band(landsat, [3,2,1]), 
                    stretch_type='percentclip', min_percent=0.1, max_percent=0.1, gamma=[1, 1, 1], dra=True)

In [ ]:
naturalcolor_sentinel = stretch(extract_band(sentinel, [3,2,1]), 
                    stretch_type='percentclip', min_percent=0.1, max_percent=0.1, gamma=[1, 1, 1], dra=True)

In [ ]:
# Image properties
import arcgis
g = arcgis.geometry.Geometry(area['extent'])

In [ ]:
samples = landsat.get_samples(g, sample_count=50,
                                 out_fields='AcquisitionDate,OBJECTID,GroupName,Category,SunAzimuth,SunElevation,CloudCover')

In [ ]:
samples[0]

In [ ]:
import datetime
value = samples[0]['attributes']['AcquisitionDate']
datetime.datetime.fromtimestamp(value /1000).strftime("Acquisition Date: %d %b, %Y")

In [ ]:
pd.DataFrame(samples[0]['attributes'], index=[0])

In [ ]:
# Query images
selected = landsat.filter_by(where="(Category = 1) AND (CloudCover <=0.50) AND (WRS_Row = 29)", 
                   geometry=arcgis.geometry.filters.intersects(area['extent']))

In [ ]:
fs = selected.query(out_fields="AcquisitionDate, GroupName, Best, CloudCover, WRS_Row, Month, Name", 
              return_geometry=True,
              return_distinct_values=False,
              order_by_fields="AcquisitionDate")

In [ ]:
df = fs.df
df

In [ ]:
df['Time'] = pd.to_datetime(df['AcquisitionDate'], unit='ms')
df['Time'].head(10)

In [ ]:
m = gis.map('Bucharest, Romania', 7)
display(m)
m.add_layer(selected.last())

In [ ]:
m = gis.map('Bucharest, Romania', 7)
display(m)
m.add_layer(selected.first())

In [ ]:
# Change detection

In [ ]:
y2000 = landsat.filter_by('OBJECTID=407250')

In [ ]:
y2014 = landsat.filter_by('OBJECTID=949422')

In [ ]:
from arcgis.raster.functions import *

In [ ]:
# diff = stretch(composite_band([ndvi(y2000, '5 4'),
#                                ndvi(y2014, '5 4'),
#                                ndvi(y2000, '5 4')]), 
#                                stretch_type='stddev',  num_stddev=3, min=0, max=255, dra=True, astype='u8')
# diff

In [ ]:
ndvi_diff = ndvi(y2014, '5 4') - ndvi(y2000, '5 4')

In [ ]:
ndvi_diff

In [ ]:
threshold_val = 0.1

In [ ]:
masked = colormap(remap(ndvi_diff, 
                        input_ranges=[threshold_val, 1], 
                        output_values=[1], 
                        no_data_ranges=[-1, threshold_val], astype='u8'), 
                  colormap=[[1, 124, 252, 0]], astype='u8')

Image(masked.export_image(bbox=area['extent'], size=[1200,450], f='image'))

In [ ]:
m.add_layer(diff)

In [ ]:
m.add_layer(masked)

In [ ]:
m

In [ ]:
lyr = masked.save('Test_viz_layer3', for_viz=True)

In [ ]:
lyr

In [ ]:
import ipywidgets as widgets

tab = widgets.Tab([m, m, m])
tab.set_title(0, 'Basemap')
tab.set_title(1, 'Natural Color')
tab.set_title(2, 'False Color')
tab